# Projeto de Melhoria de Produção no Segmento YYY

Projeto desenvolvido durante a mentoria do [João Oliveira](https://www.linkedin.com/in/joaomdeoliveira?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAZhmQ8B6jxcRnPXmY3mQ_5ESYYKgrBknvk&lipi=urn%3Ali%3Apage%3Ad_flagship3_search_srp_all%3BVDoUfGetQ%2BGy608vhivUiA%3D%3D)

Neste projeto, será abordado:

- O desafio de atingir metas de produção no segmento YYY da empresa AutoWerk.
- Análise de dados para identificar padrões de sucesso.
- Estratégias propostas para aumentar a eficiência e qualidade da produção.


**Nota:** Os dados utilizados neste notebook são fictícios.

**Por:** [Yara Rufino](https://www.linkedin.com/in/yara-de-oliveira-rufino/)

In [1]:
#bibliotecas
import pandas as pd
from datetime import date, timedelta
import datetime
import numpy as np
import os
from openpyxl import Workbook

In [2]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.9


##  <font color=red> 1.Carregar dados

>Planilha com dados de OEE geral

In [20]:
#abrir o data frame
df = pd.read_excel(r"C:\Users\Usuário\Documents\Cursos\EnvolveData\Mentoria2\Gestão da Produção\Produção AW.xlsx", sheet_name = 'DADOS OEE')
df.head(3)

,DATA,CW,TURNO,LINHA,PN,PRODUTO,Process Group,Channel Key 3,TEMPO SAP,TEMPO REAL,DIFERENÇA,TOTAL PRODUZIDO,PERFORMANCE,Parts Produced without Cycle
0,2023-01-03 00:00:00,1,1T,MK L2,75,X9-Z000-0075,1,--,18,11,-7,1,163.64,0
1,2023-01-03 00:00:00,1,2T,MK L2,75,X9-Z000-0075,1,--,18,12.5,-5.5,4,144,0
2,2023-01-03 00:00:00,1,2T,MK L1,77,X9-Z000-0077,1,--,12,11.17,-0.83,1402,107.47,3


In [21]:
#qtde de linhas e colunas
df.shape

(5911, 14)

In [22]:
#descrição das colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5911 entries, 0 to 5910
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   DATA                          5911 non-null   object
 1   CW                            5911 non-null   int64 
 2   TURNO                         5911 non-null   object
 3   LINHA                         5911 non-null   object
 4   PN                            5902 non-null   object
 5   PRODUTO                       5911 non-null   object
 6   Process Group                 5911 non-null   int64 
 7   Channel Key 3                 5911 non-null   object
 8   TEMPO SAP                     5911 non-null   object
 9   TEMPO REAL                    5911 non-null   object
 10  DIFERENÇA                     5911 non-null   object
 11  TOTAL PRODUZIDO               5911 non-null   int64 
 12  PERFORMANCE                   5911 non-null   object
 13  Parts Produced wit

In [23]:
df.isnull().sum()

DATA                            0
CW                              0
TURNO                           0
LINHA                           0
PN                              9
PRODUTO                         0
Process Group                   0
Channel Key 3                   0
TEMPO SAP                       0
TEMPO REAL                      0
DIFERENÇA                       0
TOTAL PRODUZIDO                 0
PERFORMANCE                     0
Parts Produced without Cycle    0
dtype: int64

In [24]:
#convertendo a coluna data para formato data
df['DATA'] = pd.to_datetime(df['DATA'])
df.tail()

C:\Users\Usuário\AppData\Local\Temp\ipykernel_13448\3482867533.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['DATA'] = pd.to_datetime(df['DATA'])


,DATA,CW,TURNO,LINHA,PN,PRODUTO,Process Group,Channel Key 3,TEMPO SAP,TEMPO REAL,DIFERENÇA,TOTAL PRODUZIDO,PERFORMANCE,Parts Produced without Cycle
5906,2023-12-08,32,3T,MRO 10Z,37,2017711110200,1,--,23.1,27.940789,4.840789,456,82.674829,5
5907,2023-12-08,32,3T,MRO 10Z,40,2017711114200,1,--,24.3,28.309211,4.009211,304,85.837788,2
5908,2023-12-08,32,3T,MRO 20Y,39/47,2017711111200,1,--,16.38,15.660537,-0.719463,1414,104.594111,18
5909,2023-12-08,32,3T,MRO 20Z,37,2017711110200,1,--,23.1,30.87156,7.77156,436,74.826152,3
5910,2023-12-08,32,3T,MRO 20Z,40,2017711114200,1,--,24.3,30.543796,6.243796,274,79.557892,1


## <font color=red> 2.Tratamento de dados
> Excluir colunas que não serão usadas, alterar o tipo das colunas

In [25]:
#converter as colunas Tempo real e sap e DIFERENÇA para numerico
df['TEMPO REAL'] = pd.to_numeric(df['TEMPO REAL'],errors = 'coerce')
df['TEMPO SAP'] = pd.to_numeric(df['TEMPO SAP'],errors = 'coerce')
df['DIFERENÇA'] = pd.to_numeric(df['DIFERENÇA'],errors = 'coerce')
df['PERFORMANCE'] = pd.to_numeric(df['PERFORMANCE'],errors = 'coerce')

In [26]:
#Resumo estatístico
df.describe()

,CW,Process Group,TEMPO SAP,TEMPO REAL,DIFERENÇA,TOTAL PRODUZIDO,PERFORMANCE,Parts Produced without Cycle
count,5911.000000,5911.000000,5907.000000,5868.000000,5868.000000,5911.000000,5876.000000,5911.000000
mean,16.447640,4.048215,16.376176,19.492156,3.109267,1019.535273,90.315660,8.155304
std,9.239358,16.813133,7.236696,10.949362,7.258814,779.946994,26.888489,53.108062
min,1.000000,1.000000,0.000000,0.030000,-20.715121,0.000000,0.000000,0.000000
25%,9.000000,1.000000,12.000000,12.410000,0.004486,478.500000,82.190000,3.000000
50%,17.000000,1.000000,15.000000,15.926613,1.184559,893.000000,92.618276,6.000000
75%,24.000000,1.000000,18.930000,25.251149,3.830833,1452.000000,99.950000,9.000000
max,32.000000,100.000000,40.000000,120.000000,107.560000,7592.000000,1037.000000,2984.000000


In [27]:
#excluir coluna "Channel Key 3" E "Process Group"
df.drop(['Channel Key 3'], axis = 1, inplace = True)
df.drop(['Process Group'], axis = 1, inplace = True)
df.head(3)

,DATA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO SAP,TEMPO REAL,DIFERENÇA,TOTAL PRODUZIDO,PERFORMANCE,Parts Produced without Cycle
0,2023-01-03,1,1T,MK L2,75,X9-Z000-0075,18.0,11.00,-7.00,1,163.64,0
1,2023-01-03,1,2T,MK L2,75,X9-Z000-0075,18.0,12.50,-5.50,4,144.00,0
2,2023-01-03,1,2T,MK L1,77,X9-Z000-0077,12.0,11.17,-0.83,1402,107.47,3


## <font color=red> 3.Manipulação de Dados
>Ir direto ao alvo: Segmento YYY

In [28]:
# Obter a lista de nomes únicos das linhas: MK L1,L2 e L3 pertencem ao Segmento YYY
lista_linha = df['LINHA'].unique().tolist()
print(lista_linha)

['MK L2', 'MK L1', 'MRO 20Z', 'MRO 10Z', 'SNC', 'MRO 10Y', 'MRO 20Y', 'AWZ', 'AWZ Q100', 'MK L3', 'BKL', 'IZ500', 'FU5']


In [29]:
#Não tem uma coluna de segmento, então criar uma através de 'Procv' 
condicoes = [(df['LINHA'] == 'MK L1'), (df['LINHA'] == 'MK L2'), (df['LINHA'] == 'MK L3')]

opcoes = ['YYY','YYY','YYY']

# Aplicar o procv para criar a coluna 'Segmento'
df['Segmento'] = np.select(condicoes, opcoes)
df.head(3)

,DATA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO SAP,TEMPO REAL,DIFERENÇA,TOTAL PRODUZIDO,PERFORMANCE,Parts Produced without Cycle,Segmento
0,2023-01-03,1,1T,MK L2,75,X9-Z000-0075,18.0,11.00,-7.00,1,163.64,0,YYY
1,2023-01-03,1,2T,MK L2,75,X9-Z000-0075,18.0,12.50,-5.50,4,144.00,0,YYY
2,2023-01-03,1,2T,MK L1,77,X9-Z000-0077,12.0,11.17,-0.83,1402,107.47,3,YYY


In [30]:
# Criar um dataframe apenas para o YYY
yyy =  df[df['Segmento'] == 'YYY']
display(yyy)

,DATA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO SAP,TEMPO REAL,DIFERENÇA,TOTAL PRODUZIDO,PERFORMANCE,Parts Produced without Cycle,Segmento
0,2023-01-03,1,1T,MK L2,75,X9-Z000-0075,18.0,11.000000,-7.000000,1,163.640000,0,YYY
1,2023-01-03,1,2T,MK L2,75,X9-Z000-0075,18.0,12.500000,-5.500000,4,144.000000,0,YYY
2,2023-01-03,1,2T,MK L1,77,X9-Z000-0077,12.0,11.170000,-0.830000,1402,107.470000,3,YYY
4,2023-01-03,1,1T,MK L2,77,X9-Z000-0077,13.5,12.830000,-0.670000,1513,105.200000,5,YYY
5,2023-01-03,1,2T,MK L2,77,X9-Z000-0077,13.5,12.900000,-0.600000,177,104.620000,15,YYY
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5887,2023-11-08,32,3T,MK L2,80,X9-Z000-0080,12.0,12.524490,0.524490,490,95.812286,11,YYY
5888,2023-11-08,32,3T,MK L3,62,W5-Y000-0062,10.8,11.986750,1.186750,1283,90.099486,5,YYY
5902,2023-12-08,32,3T,MK L2,61,W5-Y000-0061,11.8,10.134975,-1.665025,2193,116.428507,8,YYY
5903,2023-12-08,32,3T,MK L3,75,X9-Z000-0075,15.0,14.147287,-0.852713,258,106.027397,0,YYY


In [31]:
# Mapear os valores numericos dos dias da semana para pt-Br
dia_semana_map = {
    0: 'Segunda-feira',
    1: 'Terça-feira',
    2: 'Quarta-feira',
    3: 'Quinta-feira',
    4: 'Sexta-feira',
    5: 'Sábado',
    6: 'Domingo'
}

# extraindo o dia da semana
yyy.loc[:, 'Dia_Semana'] = yyy['DATA'].dt.dayofweek.map(dia_semana_map)
yyy

C:\Users\Usuário\AppData\Local\Temp\ipykernel_13448\665086659.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yyy.loc[:, 'Dia_Semana'] = yyy['DATA'].dt.dayofweek.map(dia_semana_map)


,DATA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO SAP,TEMPO REAL,DIFERENÇA,TOTAL PRODUZIDO,PERFORMANCE,Parts Produced without Cycle,Segmento,Dia_Semana
0,2023-01-03,1,1T,MK L2,75,X9-Z000-0075,18.0,11.000000,-7.000000,1,163.640000,0,YYY,Terça-feira
1,2023-01-03,1,2T,MK L2,75,X9-Z000-0075,18.0,12.500000,-5.500000,4,144.000000,0,YYY,Terça-feira
2,2023-01-03,1,2T,MK L1,77,X9-Z000-0077,12.0,11.170000,-0.830000,1402,107.470000,3,YYY,Terça-feira
4,2023-01-03,1,1T,MK L2,77,X9-Z000-0077,13.5,12.830000,-0.670000,1513,105.200000,5,YYY,Terça-feira
5,2023-01-03,1,2T,MK L2,77,X9-Z000-0077,13.5,12.900000,-0.600000,177,104.620000,15,YYY,Terça-feira
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5887,2023-11-08,32,3T,MK L2,80,X9-Z000-0080,12.0,12.524490,0.524490,490,95.812286,11,YYY,Quarta-feira
5888,2023-11-08,32,3T,MK L3,62,W5-Y000-0062,10.8,11.986750,1.186750,1283,90.099486,5,YYY,Quarta-feira
5902,2023-12-08,32,3T,MK L2,61,W5-Y000-0061,11.8,10.134975,-1.665025,2193,116.428507,8,YYY,Sexta-feira
5903,2023-12-08,32,3T,MK L3,75,X9-Z000-0075,15.0,14.147287,-0.852713,258,106.027397,0,YYY,Sexta-feira


In [32]:
# Média de tempo sap, tempo real e total produzido por Linha e turno
media_agregacao = yyy[['TEMPO SAP','TEMPO REAL','TOTAL PRODUZIDO','DIFERENÇA','PERFORMANCE','LINHA','TURNO',]]
media_turno_linha = media_agregacao.groupby(["LINHA","TURNO"], as_index=True)
media_turno_linha.aggregate(np.mean).round(2)

TEMPO SAP  TEMPO REAL  TOTAL PRODUZIDO  DIFERENÇA  PERFORMANCE
LINHA TURNO                                                                
MK L1 1T         12.14       14.57          1194.90       2.43        86.27
      2T         12.14       13.49          1127.71       1.36        95.72
      3T         12.20       14.07           863.75       1.87        90.38
MK L2 1T         12.14       13.01          1103.95       0.87        97.66
      2T         12.16       12.24           981.63       0.08       100.17
      3T         12.01       12.33           819.91       0.31       100.80
MK L3 1T         12.90       13.41          1061.15       0.50        97.19
      2T         12.87       13.03          1113.31       0.16        99.86
      3T         12.95       12.90           869.68      -0.03       100.75

In [33]:
#abrir o dataframe com as datas de melhor desempenho de YYY
yyy_15mil = pd.read_excel(r"C:\Users\Usuário\Documents\Cursos\EnvolveData\Mentoria2\Projeto\YYY_15mil\Analise_YYY.xlsx", sheet_name = 'Producao_YYY_15mil_detalhes')
yyy_15mil.head()

,DIA,Segmento,LINHA,TURNO,PRODUÇÃO,FALHAS,Modo_de_Falha,META,Semana,FPY,Dia_Semana,Total_Peças_Boas
0,13/01/2023,YYY,MK 01,1T,2000,13,7 CIRCUITO ABERTO / 6 FALHA NO CORTE DO PINO /...,2640,2,99.35,Sexta-feira,1987
1,18/01/2023,YYY,MK 01,1T,2350,1,1 polaridade invertida / 2 circuito aberto /,2640,3,99.96,Quarta-feira,2349
2,24/01/2023,YYY,MK 01,1T,2284,9,0,2640,4,99.61,Terça-feira,2275
3,25/01/2023,YYY,MK 01,1T,2643,1,1 circuito aberto /,2640,4,99.96,Quarta-feira,2642
4,26/01/2023,YYY,MK 01,1T,2270,1,1 circuito aberto /,2640,4,99.96,Quinta-feira,2269


In [34]:
yyy_15mil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DIA               370 non-null    object 
 1   Segmento          370 non-null    object 
 2   LINHA             370 non-null    object 
 3   TURNO             370 non-null    object 
 4   PRODUÇÃO          370 non-null    int64  
 5   FALHAS            370 non-null    int64  
 6   Modo_de_Falha     370 non-null    object 
 7   META              370 non-null    int64  
 8   Semana            370 non-null    int64  
 9   FPY               370 non-null    float64
 10  Dia_Semana        370 non-null    object 
 11  Total_Peças_Boas  370 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 34.8+ KB


In [35]:
#convertendo a coluna data para formato data
yyy_15mil['DIA'] = pd.to_datetime(yyy_15mil['DIA'], format='%d/%m/%Y')
yyy_15mil.tail()

,DIA,Segmento,LINHA,TURNO,PRODUÇÃO,FALHAS,Modo_de_Falha,META,Semana,FPY,Dia_Semana,Total_Peças_Boas
365,2023-07-28,YYY,MK 03,3T,1775,15,2 pino recuado/17 falha laser/8 circuito abert...,1800,30,99.15,Sexta-feira,1760
366,2023-08-04,YYY,MK 03,3T,1816,16,2 pino recuado/28 falha laser/3 pin/1 capacita...,1800,31,99.12,Sexta-feira,1800
367,2023-08-09,YYY,MK 03,3T,1009,9,8 capacitance/13 falha laser/1 circuito aberto...,1350,32,99.11,Quarta-feira,1000
368,2023-08-11,YYY,MK 03,3T,1761,11,17 falha laser/10 pin/7 capacitance/1 corrente...,1800,32,99.38,Sexta-feira,1750
369,2023-08-16,YYY,MK 03,3T,882,8,8 pin/12 capacitania/15 falha laser/1 numero d...,1350,33,99.09,Quarta-feira,874


In [36]:
# Extrair as datas que atingiram produção maior ou igual a 15 mil
datas_maior_15mil = yyy_15mil['DIA'].tolist()

# Filtrar o dataframe original usando as datas obtidas
df_filtrado = yyy[yyy['DATA'].dt.date.isin([d.date() for d in datas_maior_15mil])]

# Exibir o dataframe filtrado
df_filtrado

,DATA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO SAP,TEMPO REAL,DIFERENÇA,TOTAL PRODUZIDO,PERFORMANCE,Parts Produced without Cycle,Segmento,Dia_Semana
306,2023-01-13,2,2T,MK L2,63,W5-Y000-0063,11.8,10.090000,-1.710000,2406,116.980000,12,YYY,Sexta-feira
308,2023-01-13,2,1T,MK L2,62,W5-Y000-0062,11.8,10.680000,-1.120000,2370,110.520000,11,YYY,Sexta-feira
311,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,12.0,11.680000,-0.320000,1882,102.780000,6,YYY,Sexta-feira
313,2023-01-13,2,2T,MK L3,77,X9-Z000-0077,12.0,11.730000,-0.270000,2317,102.320000,12,YYY,Sexta-feira
314,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,12.0,11.750000,-0.250000,593,102.140000,16,YYY,Sexta-feira
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5637,2023-02-08,31,1T,MK L2,62,W5-Y000-0062,11.8,12.303203,0.503203,1405,95.909985,7,YYY,Quarta-feira
5638,2023-02-08,31,1T,MK L3,77,X9-Z000-0077,12.0,13.229508,1.229508,122,90.706320,4,YYY,Quarta-feira
5639,2023-02-08,31,1T,MK L3,79,X9-Z000-0079,12.0,12.949686,0.949686,159,92.666343,0,YYY,Quarta-feira
5643,2023-02-08,31,2T,MK L2,75,X9-Z000-0075,18.0,18.485895,0.485895,1737,97.371535,1,YYY,Quarta-feira


## <font color=red> 4.Fazer conexão com um SQL
>Salvar os dados tratados para ser utilizado em um Dataviz

> Criando tabela no sql

```


CREATE TABLE YYY_OEE(
	 DIA DATE,
	 CW INT,
	 TURNO VARCHAR (2),
	 LINHA VARCHAR (20),
	 PN VARCHAR (15),
	 PRODUTO VARCHAR (20),
	 TEMPO_SAP FLOAT,
	 TEMPO_REAL FLOAT,
	 DIFERENÇA FLOAT,
	 TOTAL_PRODUZIDO INT,
	 PERFORMANCE FLOAT,
	 Parts_Produced_without_Cycle INT,
	 Segmento VARCHAR (10),
	 Dia_Semana VARCHAR (30)

)
```

In [37]:
df_filtrado.columns

Index(['DATA', 'CW', 'TURNO', 'LINHA', 'PN', 'PRODUTO', 'TEMPO SAP',
       'TEMPO REAL', 'DIFERENÇA', 'TOTAL PRODUZIDO', 'PERFORMANCE',
       'Parts Produced without Cycle', 'Segmento', 'Dia_Semana'],
      dtype='object')

In [39]:
#renomear a coluna DATA pois é uma palavra reservada no SQL e as outras por conta do espaço e o SQL não reconhece
df_filtrado =  df_filtrado.rename(columns={'DATA': 'DIA'})
df_filtrado =  df_filtrado.rename(columns={'TEMPO SAP': 'TEMPO_SAP'})
df_filtrado =  df_filtrado.rename(columns={'TEMPO REAL': 'TEMPO_REAL'})
df_filtrado =  df_filtrado.rename(columns={'Parts Produced without Cycle': 'Parts_Produced_without_Cycle'})
df_filtrado =  df_filtrado.rename(columns={'TOTAL PRODUZIDO': 'TOTAL_PRODUZIDO'})
df_filtrado.tail()

,DIA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO_SAP,TEMPO_REAL,DIFERENÇA,TOTAL_PRODUZIDO,PERFORMANCE,Parts_Produced_without_Cycle,Segmento,Dia_Semana
5637,2023-02-08,31,1T,MK L2,62,W5-Y000-0062,11.8,12.303203,0.503203,1405,95.909985,7,YYY,Quarta-feira
5638,2023-02-08,31,1T,MK L3,77,X9-Z000-0077,12.0,13.229508,1.229508,122,90.706320,4,YYY,Quarta-feira
5639,2023-02-08,31,1T,MK L3,79,X9-Z000-0079,12.0,12.949686,0.949686,159,92.666343,0,YYY,Quarta-feira
5643,2023-02-08,31,2T,MK L2,75,X9-Z000-0075,18.0,18.485895,0.485895,1737,97.371535,1,YYY,Quarta-feira
5645,2023-02-08,31,3T,MK L2,80,X9-Z000-0080,12.0,12.438503,0.438503,1122,96.474635,4,YYY,Quarta-feira


In [40]:
# Biblioteca que conecta o Python ao SQL
import pyodbc

In [41]:
# Com autenticação do Windows
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost', 
                      database = 'Analise_YYY_OEE')

query = '''
    select 
        * 
    from YYY_OEE
'''
sql_query = pd.read_sql_query(query,conn)
sql_query

C:\Users\Usuário\AppData\Local\Temp\ipykernel_13448\318830656.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_query = pd.read_sql_query(query,conn)


,DIA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO_SAP,TEMPO_REAL,DIFERENÇA,TOTAL_PRODUZIDO,PERFORMANCE,Parts_Produced_without_Cycle,Segmento,Dia_Semana
0,2023-01-13,2,2T,MK L2,63,W5-Y000-0063,0.0,0.0,-1.710000,0,116.980000,0,YYY,Sexta-feira
1,2023-01-13,2,1T,MK L2,62,W5-Y000-0062,0.0,0.0,-1.120000,0,110.520000,0,YYY,Sexta-feira
2,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,0.0,0.0,-0.320000,0,102.780000,0,YYY,Sexta-feira
3,2023-01-13,2,2T,MK L3,77,X9-Z000-0077,0.0,0.0,-0.270000,0,102.320000,0,YYY,Sexta-feira
4,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,0.0,0.0,-0.250000,0,102.140000,0,YYY,Sexta-feira
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,2023-02-08,31,1T,MK L2,62,W5-Y000-0062,0.0,0.0,0.503203,0,95.909985,0,YYY,Quarta-feira
441,2023-02-08,31,1T,MK L3,77,X9-Z000-0077,0.0,0.0,1.229508,0,90.706320,0,YYY,Quarta-feira
442,2023-02-08,31,1T,MK L3,79,X9-Z000-0079,0.0,0.0,0.949686,0,92.666343,0,YYY,Quarta-feira
443,2023-02-08,31,2T,MK L2,75,X9-Z000-0075,0.0,0.0,0.485895,0,97.371535,0,YYY,Quarta-feira


## Função para limpar a tabela

def SQLTruncate(NomeTabela):

    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Analise_YYY_OEE')

    cursor = conn.cursor()

    cursor.execute(f'''

                   TRUNCATE TABLE {NomeTabela}

                   ''')

    conn.commit()
    cursor.close()

## Limpando dados da tabela
SQLTruncate('YYY_OEE')

## Visualizando a tabela
SQLSelect('select *from YYY_OEE')

In [56]:
#copiar as colunas
df_filtrado.columns

Index(['DIA', 'CW', 'TURNO', 'LINHA', 'PN', 'PRODUTO', 'TEMPO_SAP',
       'TEMPO_REAL', 'DIFERENÇA', 'TOTAL_PRODUZIDO', 'PERFORMANCE',
       'Parts_Produced_without_Cycle', 'Segmento', 'Dia_Semana'],
      dtype='object')

In [46]:
# Criar uma base
base = pd.DataFrame(columns=['DIA', 'CW', 'TURNO', 'LINHA', 'PN', 'PRODUTO', 'TEMPO_SAP',
       'TEMPO_REAL', 'DIFERENÇA', 'TOTAL_PRODUZIDO', 'PERFORMANCE',
       'Parts_Produced_without_Cycle', 'Segmento', 'Dia_Semana'])
base

,DIA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO_SAP,TEMPO_REAL,DIFERENÇA,TOTAL_PRODUZIDO,PERFORMANCE,Parts_Produced_without_Cycle,Segmento,Dia_Semana


In [47]:
# "Combinamos" as bases
base = base.append(df_filtrado)
base

C:\Users\Usuário\AppData\Local\Temp\ipykernel_13448\1900034503.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append(df_filtrado)


,DIA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO_SAP,TEMPO_REAL,DIFERENÇA,TOTAL_PRODUZIDO,PERFORMANCE,Parts_Produced_without_Cycle,Segmento,Dia_Semana
306,2023-01-13,2,2T,MK L2,63,W5-Y000-0063,11.8,10.090000,-1.710000,2406,116.980000,12,YYY,Sexta-feira
308,2023-01-13,2,1T,MK L2,62,W5-Y000-0062,11.8,10.680000,-1.120000,2370,110.520000,11,YYY,Sexta-feira
311,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,12.0,11.680000,-0.320000,1882,102.780000,6,YYY,Sexta-feira
313,2023-01-13,2,2T,MK L3,77,X9-Z000-0077,12.0,11.730000,-0.270000,2317,102.320000,12,YYY,Sexta-feira
314,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,12.0,11.750000,-0.250000,593,102.140000,16,YYY,Sexta-feira
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5637,2023-02-08,31,1T,MK L2,62,W5-Y000-0062,11.8,12.303203,0.503203,1405,95.909985,7,YYY,Quarta-feira
5638,2023-02-08,31,1T,MK L3,77,X9-Z000-0077,12.0,13.229508,1.229508,122,90.706320,4,YYY,Quarta-feira
5639,2023-02-08,31,1T,MK L3,79,X9-Z000-0079,12.0,12.949686,0.949686,159,92.666343,0,YYY,Quarta-feira
5643,2023-02-08,31,2T,MK L2,75,X9-Z000-0075,18.0,18.485895,0.485895,1737,97.371535,1,YYY,Quarta-feira


In [48]:
# Excluir valores vazios caso tenha
base.fillna('')

,DIA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO_SAP,TEMPO_REAL,DIFERENÇA,TOTAL_PRODUZIDO,PERFORMANCE,Parts_Produced_without_Cycle,Segmento,Dia_Semana
306,2023-01-13,2,2T,MK L2,63,W5-Y000-0063,11.8,10.090000,-1.710000,2406,116.980000,12,YYY,Sexta-feira
308,2023-01-13,2,1T,MK L2,62,W5-Y000-0062,11.8,10.680000,-1.120000,2370,110.520000,11,YYY,Sexta-feira
311,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,12.0,11.680000,-0.320000,1882,102.780000,6,YYY,Sexta-feira
313,2023-01-13,2,2T,MK L3,77,X9-Z000-0077,12.0,11.730000,-0.270000,2317,102.320000,12,YYY,Sexta-feira
314,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,12.0,11.750000,-0.250000,593,102.140000,16,YYY,Sexta-feira
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5637,2023-02-08,31,1T,MK L2,62,W5-Y000-0062,11.8,12.303203,0.503203,1405,95.909985,7,YYY,Quarta-feira
5638,2023-02-08,31,1T,MK L3,77,X9-Z000-0077,12.0,13.229508,1.229508,122,90.706320,4,YYY,Quarta-feira
5639,2023-02-08,31,1T,MK L3,79,X9-Z000-0079,12.0,12.949686,0.949686,159,92.666343,0,YYY,Quarta-feira
5643,2023-02-08,31,2T,MK L2,75,X9-Z000-0075,18.0,18.485895,0.485895,1737,97.371535,1,YYY,Quarta-feira


In [49]:
# Função para criar um banco de dados SQL
def SQLInsert(df_filtrado):
    
    base = pd.DataFrame(columns=['DIA', 'CW', 'TURNO', 'LINHA', 'PN', 'PRODUTO', 'TEMPO_SAP','TEMPO_REAL', 'DIFERENÇA', 
        'TOTAL_PRODUZIDO', 'PERFORMANCE','Parts_Produced_without_Cycle', 'Segmento', 'Dia_Semana'])
    df_filtrado = base.append(df_filtrado).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Analise_YYY_OEE')

    cursor = conn.cursor()

    for index, row in df_filtrado.iterrows():

        cursor.execute('''
            INSERT INTO YYY_OEE(
                DIA,
                CW,
                TURNO,
                LINHA,
                PN,
                PRODUTO,
                TEMPO_SAP,
                TEMPO_REAL,
                DIFERENÇA,
                TOTAL_PRODUZIDO,
                PERFORMANCE,
                Parts_Produced_without_Cycle,
                Segmento,
                Dia_Semana
                
            ) 
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        ''', 
        row['DIA'], 
        row['CW'], 
        row['TURNO'], 
        row['LINHA'], 
        row['PN'], 
        row['PRODUTO'], 
        row['TEMPO_SAP'], 
        row['TEMPO_REAL'],
        row['DIFERENÇA'],
        row['TOTAL_PRODUZIDO'],   
        row['PERFORMANCE'],
        row['Parts_Produced_without_Cycle'],
        row['Segmento'],
        row['Dia_Semana'], 
        )


    conn.commit()
    cursor.close()

In [50]:
#Inserindo os dados
SQLInsert(base)

C:\Users\Usuário\AppData\Local\Temp\ipykernel_13448\2139757426.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_filtrado = base.append(df_filtrado).fillna('')


In [51]:
# Função para executar consultas SQL
def SQLSelect(query):
    
    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Analise_YYY_OEE')

    out = pd.read_sql_query(query,conn)
    return out

In [52]:
SQLSelect('''
    select 
        *
    from YYY_OEE
''')

C:\Users\Usuário\AppData\Local\Temp\ipykernel_13448\184355721.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  out = pd.read_sql_query(query,conn)


,DIA,CW,TURNO,LINHA,PN,PRODUTO,TEMPO_SAP,TEMPO_REAL,DIFERENÇA,TOTAL_PRODUZIDO,PERFORMANCE,Parts_Produced_without_Cycle,Segmento,Dia_Semana
0,2023-01-13,2,2T,MK L2,63,W5-Y000-0063,11.8,10.090000,-1.710000,2406,116.980000,12,YYY,Sexta-feira
1,2023-01-13,2,1T,MK L2,62,W5-Y000-0062,11.8,10.680000,-1.120000,2370,110.520000,11,YYY,Sexta-feira
2,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,12.0,11.680000,-0.320000,1882,102.780000,6,YYY,Sexta-feira
3,2023-01-13,2,2T,MK L3,77,X9-Z000-0077,12.0,11.730000,-0.270000,2317,102.320000,12,YYY,Sexta-feira
4,2023-01-13,2,1T,MK L3,77,X9-Z000-0077,12.0,11.750000,-0.250000,593,102.140000,16,YYY,Sexta-feira
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,2023-02-08,31,1T,MK L2,62,W5-Y000-0062,11.8,12.303203,0.503203,1405,95.909985,7,YYY,Quarta-feira
441,2023-02-08,31,1T,MK L3,77,X9-Z000-0077,12.0,13.229508,1.229508,122,90.706320,4,YYY,Quarta-feira
442,2023-02-08,31,1T,MK L3,79,X9-Z000-0079,12.0,12.949686,0.949686,159,92.666343,0,YYY,Quarta-feira
443,2023-02-08,31,2T,MK L2,75,X9-Z000-0075,18.0,18.485895,0.485895,1737,97.371535,1,YYY,Quarta-feira


In [53]:
#Nome da pasta a ser criada
nome_pasta = r'C:\Users\Usuário\Documents\Cursos\EnvolveData\Mentoria2\Projeto\YYY_15mil'

# Verificar se a pasta não existe e, se não existir, criar
if not os.path.exists(nome_pasta):
    os.makedirs(nome_pasta)
    
#criar o caminho para o arquivo ser salvo    
caminho_arquivo = r'C:\Users\Usuário\Documents\Cursos\EnvolveData\Mentoria2\Projeto\YYY_15mil\Analise_YYY_OEE.xlsx'

In [55]:
# Crie um objeto ExcelWriter usando o arquivo como destino
arq_excel = pd.ExcelWriter(caminho_arquivo, engine='openpyxl')

#Salve o DataFrame 4 na segunda aba
yyy.to_excel(arq_excel, sheet_name='Producao_YYY_OEE', index=False)

# Salve e feche o arquivo Excel
arq_excel.save()
arq_excel.close()

print("DataFrame salvo com sucesso")

C:\Users\Usuário\AppData\Local\Temp\ipykernel_13448\2342841416.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  arq_excel.save()


DataFrame salvo com sucesso
